<a href="https://colab.research.google.com/github/SergeiVKalinin/MSE_Spring2025/blob/main/Module_2/8_StatisticalModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook for the MSE510/494 course at MSE UTK
- Instructor Sergei V. Kalinin

# Diffusion Limited Aggregation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# Define grid size and parameters
GRID_SIZE = (48, 48)
NUM_PARTICLES = 300  # Total number of particles to be launched
STICKING_PROBABILITY = 1  # Probability that a particle sticks upon contact
MAX_DIFFUSION_STEPS = 500  # Prevents infinite wandering
SAVE_DIR = "dle_fractal_frames"

# Create directory to save images
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Initialize grid with a single seed at the center
def initialize_grid(size):
    grid = np.zeros(size, dtype=int)
    center = (size[0]//2, size[1]//2)
    grid[center] = 1
    return grid

# Function to release a particle from the edge
def release_particle(grid_size):
    edge = np.random.choice([0, 1, 2, 3])  # Select a random edge (top, bottom, left, right)
    if edge == 0:
        return 0, np.random.randint(0, grid_size[1])  # Top edge
    elif edge == 1:
        return grid_size[0] - 1, np.random.randint(0, grid_size[1])  # Bottom edge
    elif edge == 2:
        return np.random.randint(0, grid_size[0]), 0  # Left edge
    else:
        return np.random.randint(0, grid_size[0]), grid_size[1] - 1  # Right edge

# Function to simulate particle movement and sticking
def diffuse_particle(grid, sticking_prob):
    x, y = release_particle(grid.shape)
    steps = 0  # Step counter to prevent infinite loops
    while steps < MAX_DIFFUSION_STEPS:
        direction = np.array([(0, 1), (1, 0), (0, -1), (-1, 0)])
        dx, dy = direction[np.random.choice(len(direction))]  # Select a random direction
        x_new, y_new = x + dx, y + dy

        # Ensure new position is within bounds
        if x_new <= 0 or x_new >= grid.shape[0] - 1 or y_new <= 0 or y_new >= grid.shape[1] - 1:
            x, y = release_particle(grid.shape)  # Restart particle if it reaches the boundary
            steps = 0  # Reset step counter
            continue

        # Stick if near aggregate with probability
        if np.any(grid[max(0, x_new-1):min(grid.shape[0], x_new+2), max(0, y_new-1):min(grid.shape[1], y_new+2)] == 1) and np.random.rand() < sticking_prob:
            grid[x_new, y_new] = 1
            return grid  # Return updated grid after sticking

        x, y = x_new, y_new  # Move particle
        steps += 1

    return grid  # Return grid even if particle doesn't stick (to prevent infinite loop)

# Initialize the grid
current_grid = initialize_grid(GRID_SIZE)

# Save each step as an image
def save_grid_as_image(grid, step):
    plt.imshow(grid, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.savefig(f"{SAVE_DIR}/step_{step:04d}.jpg", bbox_inches='tight', pad_inches=0)
    plt.close()

# Run the simulation and save images
for step in range(1, NUM_PARTICLES + 1):
    current_grid = diffuse_particle(current_grid, STICKING_PROBABILITY)
    if step % 1 == 0:  # Save an image every step
        save_grid_as_image(current_grid, step)

# Create an animated GIF
def create_gif(save_dir, output_filename):
    images = [Image.open(f"{save_dir}/step_{i:04d}.jpg") for i in range(1, NUM_PARTICLES + 1)]
    images[0].save(output_filename, save_all=True, append_images=images[1:], duration=100, loop=0)

# Generate GIF
gif_filename = "dla_fractal.gif"
create_gif(SAVE_DIR, gif_filename)

print(f"GIF saved as {gif_filename}")


GIF saved as dla_fractal.gif


# Game of Life

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
import os

# Define grid size and steps
GRID_SIZE = (200, 200)
NUM_STEPS = 500
SAVE_DIR = "game_of_life_frames"

# Create directory to save images
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Initialize grid randomly with 0s and 1s
def initialize_grid(size):
    return np.random.choice([0, 1], size=size, p=[0.8, 0.2])

# Function to update the grid based on the Game of Life rules
def update(grid):
    new_grid = grid.copy()
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            # Count live neighbors
            neighbors = np.sum(grid[max(0, i-1):min(i+2, grid.shape[0]), max(0, j-1):min(j+2, grid.shape[1])]) - grid[i, j]

            # Apply rules of the game
            if grid[i, j] == 1 and (neighbors < 2 or neighbors > 3):
                new_grid[i, j] = 0  # Underpopulation or Overpopulation
            elif grid[i, j] == 0 and neighbors == 3:
                new_grid[i, j] = 1  # Reproduction
    return new_grid

# Initialize the grid
current_grid = initialize_grid(GRID_SIZE)

# Save each step as an image
def save_grid_as_image(grid, step):
    plt.imshow(grid, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.savefig(f"{SAVE_DIR}/step_{step:03d}.jpg", bbox_inches='tight', pad_inches=0)
    plt.close()

# Run the simulation and save images
for step in range(NUM_STEPS):
    save_grid_as_image(current_grid, step)
    current_grid = update(current_grid)

# Create an animated GIF
def create_gif(save_dir, output_filename):
    images = [Image.open(f"{save_dir}/step_{i:03d}.jpg") for i in range(NUM_STEPS)]
    images[0].save(output_filename, save_all=True, append_images=images[1:], duration=100, loop=0)

# Generate GIF
gif_filename = "game_of_life.gif"
create_gif(SAVE_DIR, gif_filename)

print(f"GIF saved as {gif_filename}")

GIF saved as game_of_life.gif


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
import os

# Define grid size and simulation parameters
GRID_SIZE = (50, 50)
STEPS = 300
SAVE_DIR = "game_of_life_gifs"

# Create directory to save gifs
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Define famous Game of Life configurations
def get_configurations():
    configs = {
        "Block": [(24, 24), (24, 25), (25, 24), (25, 25)],
        "Beehive": [(24, 23), (23, 24), (23, 25), (24, 26), (25, 24), (25, 25)],
        "Blinker": [(24, 23), (24, 24), (24, 25)],
        "Toad": [(23, 24), (23, 25), (23, 26), (24, 23), (24, 24), (24, 25)],
        "Beacon": [(22, 22), (22, 23), (23, 22), (23, 23), (24, 24), (24, 25), (25, 24), (25, 25)],
        "Glider": [(24, 23), (25, 24), (23, 25), (24, 25), (25, 25)],
        "LWSS": [(22, 23), (22, 26), (23, 22), (24, 22), (25, 22), (25, 26), (26, 22), (26, 23), (26, 24), (26, 25)],
        "R-pentomino": [(24, 24), (24, 25), (25, 23), (25, 24), (26, 24)],
        "Diehard": [(24, 29), (25, 23), (25, 24), (26, 24), (26, 28), (26, 29), (26, 30)],
        "Acorn": [(24, 23), (26, 24), (23, 25), (24, 25), (26, 25), (27, 25), (28, 25)],
    }
    return configs

# Initialize grid
def initialize_grid(config):
    grid = np.zeros(GRID_SIZE, dtype=int)
    for x, y in config:
        grid[x, y] = 1
    return grid

# Update grid according to Game of Life rules
def update(grid):
    new_grid = grid.copy()
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            neighbors = np.sum(grid[max(0, i-1):min(i+2, grid.shape[0]), max(0, j-1):min(j+2, grid.shape[1])]) - grid[i, j]
            if grid[i, j] == 1 and (neighbors < 2 or neighbors > 3):
                new_grid[i, j] = 0
            elif grid[i, j] == 0 and neighbors == 3:
                new_grid[i, j] = 1
    return new_grid

# Animate the Game of Life for each configuration
def run_simulation(name, config):
    grid = initialize_grid(config)
    frames = []

    for _ in range(STEPS):
        frames.append(grid.copy())
        grid = update(grid)

    gif_filename = os.path.join(SAVE_DIR, f"{name}.gif")
    images = [Image.fromarray((frame * 255).astype(np.uint8)) for frame in frames]
    images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=50, loop=0)
    print(f"GIF saved: {gif_filename}")

# Run simulation for all configurations
configs = get_configurations()
for name, config in configs.items():
    run_simulation(name, config)

print("All GIFs saved successfully!")

GIF saved: game_of_life_gifs/Block.gif
GIF saved: game_of_life_gifs/Beehive.gif
GIF saved: game_of_life_gifs/Blinker.gif
GIF saved: game_of_life_gifs/Toad.gif
GIF saved: game_of_life_gifs/Beacon.gif
GIF saved: game_of_life_gifs/Glider.gif
GIF saved: game_of_life_gifs/LWSS.gif
GIF saved: game_of_life_gifs/R-pentomino.gif
GIF saved: game_of_life_gifs/Diehard.gif
GIF saved: game_of_life_gifs/Acorn.gif
All GIFs saved successfully!


# 1D Cellular Automata

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# Define simulation parameters
GRID_SIZE = 100  # Width of the 1D automaton
STEPS = 100  # Number of steps to simulate
SAVE_DIR = "1d_cellular_automata"

# Create directory to save images
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Function to apply a given rule
def apply_rule(rule_number, neighborhood):
    rule_bin = np.array([int(x) for x in np.binary_repr(rule_number, width=8)])
    index = int("".join(map(str, neighborhood)), 2)
    return rule_bin[7 - index]

# Function to simulate a 1D cellular automaton
def simulate_rule(rule_number):
    grid = np.zeros((STEPS, GRID_SIZE), dtype=int)
    grid[0, GRID_SIZE // 2] = 1  # Start with a single active cell in the middle

    for t in range(1, STEPS):
        for i in range(1, GRID_SIZE - 1):
            neighborhood = grid[t-1, i-1:i+2]
            grid[t, i] = apply_rule(rule_number, neighborhood)
    return grid

# Function to save each rule's evolution as an image
def save_grid_as_image(grid, rule_number):
    plt.imshow(grid, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.savefig(f"{SAVE_DIR}/rule_{rule_number:03d}.jpg", bbox_inches='tight', pad_inches=0)
    plt.close()

# Run simulation for all 256 rules
for rule in range(256):
    grid = simulate_rule(rule)
    save_grid_as_image(grid, rule)

# Create an animated GIF from saved images
def create_gif(save_dir, output_filename):
    images = [Image.open(f"{save_dir}/rule_{i:03d}.jpg") for i in range(256)]
    images[0].save(output_filename, save_all=True, append_images=images[1:], duration=100, loop=0)

# Generate GIF
gif_filename = "1d_cellular_automata.gif"
create_gif(SAVE_DIR, gif_filename)

print(f"GIF saved as {gif_filename}")


GIF saved as 1d_cellular_automata.gif


# Arbitrary rules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

# Define simulation parameters
GRID_SIZE = (50, 50)  # Grid size for the cellular automaton
STEPS = 300  # Number of steps to simulate
SAVE_DIR = "custom_game_of_life"
GIF_PATH = "custom_game_of_life.gif"

# Create directory to save images
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Initialize grid randomly
def initialize_grid():
    return np.random.choice([0, 1], size=GRID_SIZE, p=[0.8, 0.2])

# Custom rules for the Game of Life variant
def update(grid):
    new_grid = np.zeros_like(grid)
    for i in range(1, grid.shape[0] - 1):
        for j in range(1, grid.shape[1] - 1):
            # Count neighbors on main axes (N, S, E, W)
            main_axes_neighbors = (
                grid[i-1, j] + grid[i+1, j] + grid[i, j-1] + grid[i, j+1]
            )

            # Count diagonal neighbors (NE, NW, SE, SW)
            diagonal_neighbors = (
                grid[i-1, j-1] + grid[i-1, j+1] + grid[i+1, j-1] + grid[i+1, j+1]
            )

            total_neighbors = main_axes_neighbors + diagonal_neighbors

            # Apply the rules
            if grid[i, j] == 1:
                if main_axes_neighbors == 2 and diagonal_neighbors in [2, 3, 4]:
                    new_grid[i, j] = 1  # Particle survives
            elif grid[i, j] == 0:
                if total_neighbors in [3, 4, 5]:
                    new_grid[i, j] = 1  # Particle appears

    return new_grid

# Function to save grid state as an image
def save_grid_as_image(grid, step):
    image_path = f"{SAVE_DIR}/step_{step:04d}.png"
    plt.imshow(grid, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    return image_path

# Run the simulation and save images
image_paths = []
current_grid = initialize_grid()
for step in range(STEPS):
    image_path = save_grid_as_image(current_grid, step)
    image_paths.append(image_path)
    current_grid = update(current_grid)

# Create animated GIF
def create_gif(image_paths, gif_path, duration=100):
    images = [Image.open(img) for img in image_paths]
    images[0].save(gif_path, save_all=True, append_images=images[1:], duration=duration, loop=0)

create_gif(image_paths, GIF_PATH)

print("Simulation completed. Images saved in:", SAVE_DIR)
print("Animated GIF saved as:", GIF_PATH)


Simulation completed. Images saved in: custom_game_of_life
Animated GIF saved as: custom_game_of_life.gif
